In [6]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import random
import array
import tensorflow as tf
from tensorflow.keras import Model

In [7]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

Question 1

In [8]:
# get the most popular recipes
recipeCount = defaultdict(int)
totalCooked = 0
validationSet = defaultdict(int)
nums = 0
totalRecipe = set()
itemsPerUser = defaultdict(set)

for user,recipe,_ in readCSV("assignment1/trainInteractions.csv.gz"):
    if nums >= 400000:
        validationSet[(user,recipe)] = 1
    else:
        recipeCount[recipe] += 1
        totalCooked += 1
        totalRecipe.add(recipe)
        itemsPerUser[user].add(recipe)
        nums += 1
        

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()


In [9]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalCooked/2: break

In [13]:
new_validationSet = defaultdict(int)
for (u,r) in validationSet:
    diff = list(totalRecipe - itemsPerUser[u])
    uncook = random.choice(diff)
    new_validationSet[(u,uncook)] = 0
    new_validationSet[(u,r)] = 1

In [14]:
right_results = 0
for (u,r) in new_validationSet:
    if (r in return1 and new_validationSet[(u,r)] == 1) or (r not in return1 and new_validationSet[(u,r)] == 0):
        right_results += 1
    else:
        right_results += 0

In [15]:
Accuracy = right_results/len(new_validationSet)
print('The performance (accuracy) of the baseline model: %.4f' %Accuracy)

The performance (accuracy) of the baseline model: 0.6647


Question 2

In [16]:
import numpy as np

In [17]:
threshold_list = list(np.linspace(0,1,101))

In [18]:
threshold_validationSet = defaultdict()
for m in threshold_list:
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalCooked*m: break
    right_results = 0
    for (u,r) in new_validationSet:
        if (r in return1 and new_validationSet[(u,r)] == 1) or (r not in return1 and new_validationSet[(u,r)] == 0):
            right_results += 1
        else:
            right_results += 0
    Accuracy = right_results/len(new_validationSet)
    threshold_validationSet[m] = Accuracy

In [19]:
for key,value in threshold_validationSet.items():
    if(value == max(threshold_validationSet.values())):
        print((key,value))

(0.59, 0.667431995758176)


Question 3

In [20]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2)) 
    denom = len(s1.union(s2))
    return numer / denom

In [21]:
usersPerItem = defaultdict(set)
nums1 = 0
for user,recipe,_ in readCSV("assignment1/trainInteractions.csv.gz"):
    if nums1 < 400000:
        usersPerItem[recipe].add(user)
        nums1 += 1

In [22]:
def maximum_similarity(u,g):
    similarities = []
    training_recipes = itemsPerUser[u]
    for recipe in training_recipes:
        similarity = Jaccard(usersPerItem[g],usersPerItem[recipe])
        similarities.append(similarity)
    if len(similarities) == 0:
        return 0
    else:
        return max(similarities)

In [23]:
right_results = 0
threshold = 0.01
for (u,r) in new_validationSet:
    if (maximum_similarity(u,r) >= threshold and new_validationSet[(u,r)] == 1) or (maximum_similarity(u,r) < threshold and new_validationSet[(u,r)] == 0):
        right_results += 1
    else:
        right_results += 0

In [24]:
Accuracy = right_results/len(new_validationSet)
print('The performance (accuracy) of the baseline model: %.4f' %Accuracy)

The performance (accuracy) of the baseline model: 0.5682


In [25]:
sim_result = defaultdict(float)
for u,g in new_validationSet:
    sim_result[(u,g)] = maximum_similarity(u,g)

In [26]:
threshold_sim_validationSet = defaultdict()
for k in list(np.linspace(0,1,101)):
    sim_prediction = defaultdict(int)
    for (m,n) in sim_result:
        if sim_result[(m,n)] >= k:
            sim_prediction[(m,n)] = 1
        else:
            sim_prediction[(m,n)] = 0
    sim_right_result = 0
    for (p,q) in sim_prediction:
        if sim_prediction[(p,q)] == new_validationSet[(p,q)]:
            sim_right_result += 1
        else:
            sim_right_result += 0
    sim_Accuracy = sim_right_result/len(new_validationSet)
    threshold_sim_validationSet[k] = sim_Accuracy

In [27]:
for key,value in threshold_sim_validationSet.items():
    if(value == max(threshold_sim_validationSet.values())):
        print((key,value))

(0.01, 0.5682043278609802)


Question4

In [28]:
combine_validationSet = new_validationSet.copy()

In [29]:
populatity_threshold = 0.59
similarity_threshold = 0.01
return_combine = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return_combine.add(i)
  if count > totalCooked*populatity_threshold: break
combine_right_results = 0
for (u,r) in combine_validationSet:
    if (r in return_combine and maximum_similarity(u,r) >= similarity_threshold and combine_validationSet[(u,r)] == 1) or ((r not in return_combine or maximum_similarity(u,r) < similarity_threshold) and combine_validationSet[(u,r)] == 0):
        combine_right_results += 1
    else:
        combine_right_results += 0
combine_accuracy = combine_right_results/len(combine_validationSet)

In [30]:
combine_accuracy

0.6528707344157988

Question5

In [30]:
predictions = open("predictions_Made.txt", 'w')
for l in open("assignment1/stub_Made.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if (i in return_combine and maximum_similarity(u,i) >= similarity_threshold):
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

Question9

In [470]:
def readCSV1(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d['rating']

In [471]:
userIDs = {}
itemIDs = {}
interactions = []

for user,recipe, rating in readCSV1("assignment1/trainInteractions.csv.gz"):
    if not user in userIDs: userIDs[user] = len(userIDs)
    if not recipe in itemIDs: itemIDs[recipe] = len(itemIDs)
    interactions.append((user,recipe,int(rating)))

In [472]:
random.shuffle(interactions)
len(interactions)

500000

In [473]:
nTrain = int(len(interactions) * 0.9)
nValidation = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsValidation = interactions[nTrain:]

In [474]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [517]:
interactionsTrain[:10]

[('00000014', '50939497', 5),
 ('21919583', '94320955', 5),
 ('53994337', '39151782', 5),
 ('45601931', '02757863', 5),
 ('35541031', '77444570', 5),
 ('72697636', '21601840', 5),
 ('05472157', '63771765', 4),
 ('49044514', '38256576', 5),
 ('59205393', '65374177', 5),
 ('64368882', '27610964', 5)]

In [478]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [479]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [480]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [481]:
modelLFM = LatentFactorModel(mu, 3, 1)

In [482]:
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [484]:
for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 3.3557003
iteration 20, objective = 1.2718706
iteration 30, objective = 0.5391916
iteration 40, objective = 0.46692204
iteration 50, objective = 0.4973867
iteration 60, objective = 0.45403153
iteration 70, objective = 0.45464355
iteration 80, objective = 0.46259156
iteration 90, objective = 0.4431317
iteration 100, objective = 0.45313326


In [485]:
error = []
for k in range(len(interactionsValidation)):
    error.append((modelLFM.predict(userIDs[interactionsValidation[k][0]], itemIDs[interactionsValidation[k][1]]).numpy() - interactionsValidation[k][2]) **2)       
                 

In [486]:
sum(error)/len(interactionsValidation)

0.8830935668470781

Question 10

In [487]:
user_coefficient = tf.get_static_value(modelLFM.trainable_variables)[1].numpy()

In [488]:
recipe_coefficient = tf.get_static_value(modelLFM.trainable_variables)[2].numpy()

In [489]:
contant_coefficient = tf.get_static_value(modelLFM.trainable_variables)[0].numpy()

In [490]:
user_coefficient

array([-1.39616837e-04,  1.07845335e-05, -2.45239917e-05, ...,
       -3.58505313e-05,  1.66078262e-05, -2.45104839e-05], dtype=float32)

In [491]:
recipe_coefficient

array([ 3.0576572e-05, -6.4843814e-05, -2.7597243e-05, ...,
        3.8888909e-05,  2.0743835e-05,  2.2903696e-05], dtype=float32)

In [492]:
contant_coefficient

4.583514

In [493]:
print(np.where(user_coefficient == max(user_coefficient)))
print(np.where(user_coefficient == min(user_coefficient)))
print(np.where(recipe_coefficient == max(recipe_coefficient)))
print(np.where(recipe_coefficient == min(recipe_coefficient)))

(array([32]),)
(array([1079]),)
(array([1197]),)
(array([625]),)


In [494]:
users = []
for m,n in userIDs.items():
    users.append(m)
recipes = []
for m,n in itemIDs.items():
    recipes.append(m)

In [496]:
print('user_id %s has largest β, which is %.8f' % (users[32],max(user_coefficient)))
print('user_id %s has smallest β, which is %.8f' % (users[1079],min(user_coefficient)))
print('recipe_id %s has largest β, which is %.8f' % (recipes [1197],max(recipe_coefficient)))
print('recipe_id %s has smallest β, which is %.8f' % (recipes[625],min(recipe_coefficient)))

user_id 32445558 has largest β, which is 0.00180891
user_id 47394745 has smallest β, which is -0.00075586
recipe_id 78039087 has largest β, which is 0.00021212
recipe_id 48134500 has smallest β, which is -0.00038894


Question 11

In [505]:
lambda_list = list(np.linspace(0,1.5,16))
parameter_MSE = defaultdict()

for parameter in lambda_list:
    modelLFM = LatentFactorModel(mu, 3, parameter)
    for i in range(100):
        obj = trainingStep(modelLFM, interactionsTrain)
    error = []
    for k in range(len(interactionsValidation)):
        error.append((modelLFM.predict(userIDs[interactionsValidation[k][0]], itemIDs[interactionsValidation[k][1]]).numpy() - interactionsValidation[k][2]) **2)       
        MSE = sum(error)/len(interactionsValidation)
    parameter_MSE[parameter] = MSE
    print('%.3f has finished' %parameter)
    

0.000 has finished
0.100 has finished
0.200 has finished
0.300 has finished
0.400 has finished
0.500 has finished
0.600 has finished
0.700 has finished
0.800 has finished
0.900 has finished
1.000 has finished
1.100 has finished
1.200 has finished
1.300 has finished
1.400 has finished
1.500 has finished


In [506]:
parameter_MSE

defaultdict(None,
            {0.0: 1.0891953850526088,
             0.1: 0.8826298649206996,
             0.2: 0.8828579107313518,
             0.30000000000000004: 0.8829745396646076,
             0.4: 0.8831993624126229,
             0.5: 0.8830469733862273,
             0.6000000000000001: 0.8831833483889044,
             0.7000000000000001: 0.8833700493189064,
             0.8: 0.8831520837649725,
             0.9: 0.8830983293819503,
             1.0: 0.8830958948372504,
             1.1: 0.8832427753585286,
             1.2000000000000002: 0.8831522849164533,
             1.3: 0.8830889894473811,
             1.4000000000000001: 0.8832850257509783,
             1.5: 0.8831148050989813})

It seems like when λ = 0.1, the model has the lowest MSE.

In [510]:
best_modelLFM = LatentFactorModel(mu, 3, 0.1)
for i in range(100):
    obj = trainingStep(best_modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 31.607483
iteration 20, objective = 143.04257
iteration 30, objective = 74.68475
iteration 40, objective = 17.782276
iteration 50, objective = 2.444204
iteration 60, objective = 1.0546409
iteration 70, objective = 1.1220791
iteration 80, objective = 0.8010258
iteration 90, objective = 0.53845966
iteration 100, objective = 0.45863461


In [511]:
error = []
for k in range(len(interactionsValidation)):
    error.append((best_modelLFM.predict(userIDs[interactionsValidation[k][0]], itemIDs[interactionsValidation[k][1]]).numpy() - interactionsValidation[k][2]) **2)  
MSE_new = sum(error)/len(interactionsValidation)
MSE_new

0.8826100637566288

In [514]:
contant_coefficient = tf.get_static_value(best_modelLFM.trainable_variables)[0].numpy()
predictions = open("predictions_Rated.txt", 'w')
for l in open("assignment1/stub_Rated.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if u in userIDs and i in itemIDs:
    predictions.write(u + '-' + i + ',' + str(best_modelLFM.predict(userIDs[u], itemIDs[i]).numpy()) + '\n')
  else:
    predictions.write(u + '-' + i + ',' + str(contant_coefficient) + '\n')

predictions.close()